In [ ]:
import talk.config as con

% matplotlib inline

con.config_mosek()
con.config_configManager()
con.config_matplotlib()

Constructing estimators 
------------------------
https://en.wikipedia.org/wiki/Autoregressive_model

#### Thomas Schmelzer

A very common estimator is based on AR models (autoregressive)

$$R_T = \sum_{i=1}^n w_i r_{T-i}$$

Predict the (unknown) return $R_T$ using the last $n$ previous returns. **Attention**: You may want to use volatility adjusted returns, apply filters etc.
 
How to pick the $n$ free parameters in $\mathbf{w}$? (Partial) autocorrelations? 



In [ ]:
def convolution(ts, weights):
    from statsmodels.tsa.filters.filtertools import convolution_filter
    return convolution_filter(ts, weights, nsides=1)

In [ ]:
import pandas as pd

r = pd.Series([1.0, -2.0, 1.0, 1.0, 1.5, 0.0, 2.0])
weights = [2.0, 1.0]
# trendfollowing == positive weights
x=pd.DataFrame()
x["r"] = r
x["pred"] = convolution(r, weights)
x["before"] = x["pred"].shift(1)
print(x)
print(x.corr())

In [ ]:
import pandas as pd

# mean-reversion == negative weights
r = pd.Series([1.0, -2.0, 1.0, 1.0, 1.5, 0.0, 2.0])
weights = [-2.0, -1.0]
x=pd.DataFrame()
x["r"] = r
x["pred"] = convolution(r, weights)
x["before"] = x["pred"].shift(1)
print(x)
print(x.corr())

## Looking only at the last two returns might be a bit ...

Is it a good idea to have $n=200$ free parameters?



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.tsa.stattools as sts
# generate random returns
import pandas as pd
r = pd.read_csv("SPX_Index.csv", squeeze=True, index_col=0, parse_dates=True).pct_change().dropna()
# let's compute the optimal convolution!
weights = sts.pacf(r, nlags=200)
pd.Series(data=weights[1:]).plot(kind="bar")
plt.show()

In [ ]:
# The trading system! 
pos = convolution(r, weights[1:])
pos = 1e6*(pos/pos.std())
# profit = return[today] * position[yesterday]
(r*pos.shift(1)).cumsum().plot()
plt.xlabel('Time'), plt.ylabel('Profit')
plt.show()

Bias
----

We assume the weights are exponentially decaying, e.g.

$$w_i = \frac{1}{S}\lambda^i$$

where $S$ is a suitable scaling constant and $\lambda = 1-1/N$. Note that $N \neq n$.

**Everything** that is **not** an exponentially weighted moving average is **wrong**.


In [ ]:
import numpy as np

def exp_weights(m, n=100):
    x = np.power(1.0 - 1.0/m, range(1,n+1))
    S = np.linalg.norm(x)
    return x/S

pd.Series(exp_weights(m=16,n=40)).plot(kind="bar")
plt.show()

In [ ]:
import numpy as np
import pandas as pd

periods = [2,4,6,8,12,16,24,32,48,64,96,192]
# matrix of weights
W = pd.DataFrame({period : exp_weights(m=period, n=200) for period in periods}) 
W.plot()

In [ ]:
# each column of A is a convoluted return time series
A = pd.DataFrame({period : convolution(r, W[period]).shift(1) for period in periods})  

A = A.dropna(axis=0)
r = r[A.index].dropna()

A.plot()
plt.show()

(Naive) regression
-------------------

\begin{align}
\mathbf{w}^{*}=\arg\min_{\mathbf{w} \in \mathbb{R}^m}& \rVert{\mathbf{A}\mathbf{w} - \mathbf{r}}\lVert_2 
\end{align}

In [ ]:
from numpy.linalg import lstsq
# sometimes you don't need to use MOSEK :-)
weights = pd.Series(index=periods, data=lstsq(A.values, r.values)[0])
print(weights)
(W*weights).sum(axis=1).plot(kind="bar")
(W*weights).sum(axis=1).plot()
plt.show()

### Mean variation

We provide a few indicators. Avoid fast indicators. Prefer slower indicators as they induce less trading costs.
Use the mean variation of the signal (convoluted returns here)

$$f(\mathbf{x}) = \frac{1}{n}\sum{\lvert x_i - x_{i-1}\rvert}=\frac{1}{n}\rVert{\Delta \mathbf{x}}\lVert_1$$

The $i$th column of $\mathbf{A}$ has a mean variation $d_i$. We introduce the diagonal penalty matrix $\mathbf{D}$ with $D_{i,i}=d_i$.

$$\mathbf{w}^{*}=\arg\min_{\mathbf{w} \in \mathbb{R}^m} \lVert{\mathbf{Aw}-\mathbf{r}}\rVert_2 + \lambda \rVert{\mathbf{Dw}}\lVert_1$$

In [ ]:
def mean_variation(ts):
    return ts.diff().abs().mean()

d = A.apply(mean_variation)
D = np.diag(d)
    
from mosek.fusion import *
# but you could use Mosek: 
def __two_norm(model, v):
    # add variable to model for the 2-norm of the residual
    x = model.variable(1, Domain.greaterThan(0.0))
    # add the quadratic cone
    model.constraint(Expr.vstack(x,v), Domain.inQCone())
    return x


def __one_norm(model, v):
    t = model.variable(int(v.size()), Domain.greaterThan(0.0))
    model.constraint(Expr.hstack(t, v), Domain.inQCone(int(v.size()), 2))
    return Expr.sum(t)


def ar(A, D, r, lamb=0.0):

    with Model('lasso') as model:
        n = int(A.shape[1])
        # introduce the variable for the var
        x = model.variable("x", n, Domain.unbounded())
        # minimization of the conditional value at risk
        a1 = __two_norm(model, Expr.sub(Expr.mul(DenseMatrix(A),x), Expr.constTerm(r)))
        a2 = __one_norm(model, Expr.mul(DenseMatrix(D),x))
        
        model.objective(ObjectiveSense.Minimize, Expr.add(a1, Expr.mul(a2, float(lamb))))
        model.solve()

        return x.level()


In [ ]:
t_weight = dict()
for lamb in [0.0, 1.0, 2.0, 3.0, 5.0, 7.0, 9.0, 12.0, 15.0]:
    weights = pd.Series(index=periods, data=ar(A.values, D, r.values, lamb=lamb))
    print(weights.transpose())
    t_weight[lamb] = (W*weights).sum(axis=1)
    t_weight[lamb].plot(kind="bar")
    t_weight[lamb].plot()
    plt.title(lamb)
    plt.show()

In [ ]:
for lamb in sorted(t_weight.keys()):
    pos = convolution(r, t_weight[lamb])
    pos = 1e6*(pos/pos.std())
    (r*pos.shift(1)).cumsum().plot()
    plt.title(lamb)
    plt.show()

In [ ]:
d = A.apply(mean_variation)
D = np.diag(d)
weights = pd.Series(index=periods, data=ar(A.values, D, r.values, lamb=3.0))
print(weights)
# let's make the first two moving averages really expensive!

d = A.apply(mean_variation)
D = np.diag(d)
D[0,0] = 100*D[0,0]
D[1,1] = 100*D[1,1]
a = pd.Series(index=periods, data=ar(A.values, D, r.values, lamb=3.0))
print(a)

### Summary

- The problem of constructing an estimator is corresponds to tracking an index. The index is here a historic return time series. The **assets** are standard estimators. 


- Using the (mean) total variation of the signals can help to prefer slower signals rather than expensive fast signals.


- Using a penalty induced by the $1$-norm (see LARS, LASSO) it is possible to establish a ranking amongst the indicators and construct them robustly. 


- It is possible to (vertical) stack the resulting systems to find optimal weights across a group of assets.

<a href=http://localhost:8888>Back to Overview</a>